In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from causalml.inference.tree import CausalRandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [2]:
data_hedonic = pd.read_csv('/Users/paulahofmann/Documents/Coding/Online-Review/FeaturePreperation/Data_with_Features/Total_Features_hedonic.csv')
data_utilitarian = pd.read_csv('/Users/paulahofmann/Documents/Coding/Online-Review/FeaturePreperation/Data_with_Features/Total_Features_utilitarian.csv')

In [3]:
# FIltering Data 

data_hedonic = data_hedonic[data_hedonic['product_type'] == "Chocolate"]
data_utilitarian = data_utilitarian[data_utilitarian['product_type'] == "Gaming Mouse"]

In [4]:
# Coding for hedonic and utilitarian Prodcut choice
data_hedonic ["product"] = "1"
data_utilitarian ["product"] = "0"

In [5]:
# Filtering data where helpful votes are not 0
data_hedonic = data_hedonic[data_hedonic['helpful_vote'] != 0]
data_utilitarian = data_utilitarian[data_utilitarian['helpful_vote'] != 0]


In [8]:


# Define the features
features = [
    'rating', 'rating_number', 'sentiment', 'price', 'noun_count', 'adj_count', 
    'adv_count', 'word_count', 'sentence_count', 'avg_words_per_sentence', 
    'title_length', 'review_extremity', 'elapsed_time_days', 
    'image', 'year', 'month', 'day', 'hour', 'product'
]

# Assuming you have two separate dataframes: data_hedonic and data_utilitarian
# Merge the dataframes into one
combined_df = pd.concat([data_hedonic, data_utilitarian], ignore_index=True)

# Separate the features and target variable
X = combined_df[features]
y = combined_df['helpful_ratio']

# Separate data into utilitarian and hedonic products based on the 'product' column
X_utilitarian = X[X['product'] == "0"]
X_hedonic = X[X['product'] == "1"]

# Assign treatment based on the length of each dataset
treatment_utilitarian = np.zeros(shape = len(X[X['product'] == "0"]))
treatment_hedonic = np.ones(shape = len(X[X['product'] == "1"]))

# Split the data into train and test sets
X_train_u, X_test_u, y_train_u, y_test_u, treatment_train_u, treatment_test_u = train_test_split(X_utilitarian, y[y.index.isin(X_utilitarian.index)], treatment_utilitarian, test_size=0.2, random_state=42)
X_train_h, X_test_h, y_train_h, y_test_h, treatment_train_h, treatment_test_h = train_test_split(X_hedonic, y[y.index.isin(X_hedonic.index)], treatment_hedonic, test_size=0.2, random_state=42)

# Combine the training and testing data for each product type
X_train = pd.concat([X_train_u, X_train_h], ignore_index=True)
X_test = pd.concat([X_test_u, X_test_h], ignore_index=True)
y_train = pd.concat([y_train_u, y_train_h], ignore_index=True)
y_test = pd.concat([y_test_u, y_test_h], ignore_index=True)
treatment_train = treatment_train_u + treatment_train_h
treatment_test = treatment_test_u + treatment_test_h

# Train the causal forest model
cf_utilitarian = CausalRandomForestRegressor(control_name='hedonic')
cf_utilitarian.fit(X_train, y_train, treatment_train)


ValueError: operands could not be broadcast together with shapes (414,) (412,) 

In [50]:
treatment_train

['utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilitarian',
 'utilit

In [16]:
# Check the count of utilitarian and hedonic products
utilitarian_count = combined_df[combined_df['product'] == 0].shape[0]
hedonic_count = combined_df[combined_df['product'] == 1].shape[0]

print("Number of utilitarian products:", utilitarian_count)
print("Number of hedonic products:", hedonic_count)

Number of utilitarian products: 0
Number of hedonic products: 0


In [17]:
# Check the size of the filtered dataframes
print("Number of utilitarian products:", len(X_utilitarian))
print("Number of hedonic products:", len(X_hedonic))


Number of utilitarian products: 0
Number of hedonic products: 0


In [ ]:
data_hedonic = pd.read_csv('/Users/paulahofmann/Documents/Coding/Online-Review/FeaturePreperation/Data_with_Features/Total_Features_hedonic.csv')

data_utilitarian = pd.read_csv('/Users/paulahofmann/Documents/Coding/Online-Review/FeaturePreperation/Data_with_Features/Total_Features_utilitarian.csv')
